In [75]:
import girder_client, os
import pandas as pd

PAS_files = './PAS_TransplantSlideManifest.csv'
emoryRenal_files = "./emory_renal_PAS_manifest.csv"

gc = girder_client.GirderClient(apiUrl="http://transplant.digitalslidearchive.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

Login or email:  admin
Password for admin:  ········


In [2]:
PAS_df = pd.read_csv(PAS_files)
emoryRenal_df = pd.read_csv(emoryRenal_files)
leePAS_set = list(PAS_df.name)

In [5]:
## Scan the tx slide raw set
txSlidesRaw = '5cf820a91ee779b5180282c4'
print(len(allTxSlidesRaw))

6434


In [78]:
## Key fields are caseId and/or accesionNumber, stain, stainID, blockID
from pprint import pprint
import tqdm
stainID_to_stain = { "L1" : "HE", "L2": "HE", "L3": "HE", "MT":"MT","PAS":"PAS","JMS":"JMS","C4D":"C4D", "HE":"HE","TRICH":"TRICH","TRI":"TRICH"}

def scanSlidesForBaseMetadata( slideItemInfo ):
    slideMeta = slideItemInfo['meta']

    if 'stain' not in slideMeta.keys():
        ## Fix the cases where stainID is present, but stain is not..
        if 'stainID' in slideMeta.keys():
            if slideMeta['stainID'] in stainID_to_stain:
                ### Update the metadata
                
                slideMeta['stain'] = stainID_to_stain[slideMeta['stainID']]
                gc.addMetadataToItem(slideItemInfo['_id'], slideMeta)

        if ('accessionNumber' not in slideMeta.keys() ) or ( 'caseID' not in slideMeta.keys() ):
            print(slideItemInfo['name'])
    
              

In [84]:
allTxSlidesRaw = gc.get(f'resource/{txSlidesRaw}/items?limit=0&type=collection')

for s in tqdm.tqdm(allTxSlidesRaw):
    if ('.svs' in s['name']) or ('.ndpi' in s['name'] ): 
    
        slideStatus = scanSlidesForBaseMetadata( s)


100%|██████████████████████████████████| 6434/6434 [00:00<00:00, 1507437.82it/s]

S14-1897 TRICH-001.svs
S15-3644_L5 HE-001.svs
1031480.svs


In [85]:
import pandas as pd

df = pd.json_normalize(allTxSlidesRaw)

In [68]:
fldrIDtoName = {}

for fldrId in df.folderId.unique():
    if fldrId not in fldrIDtoName:
        fldrInfo = gc.getFolder(fldrId)
        fldrIDtoName[fldrId] = fldrInfo['name']

In [70]:
df['folderName'] = df['folderId'].map(fldrIDtoName)

In [86]:
df.to_excel("allTxSlideMeta.xlsx")

In [73]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 7.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
pasStainID = gc.listItem('5e74d90608e43602a864db55')
addlPASslides = []

for x in pasStainID:
    if x['name'] not in leePAS_set:
        addlPASslides.append(x)


In [11]:
len(leePAS_set)

752

In [7]:
# for i in addlPASslides:
#     print(i['name'])
#     gc.post(f'item/{i["_id"]}/copy?folderId=657344581d7b68d83c12d049')

1019366.svs
1019431.svs
1019639.svs
1019644.svs
1020250.svs
1021356.svs
1021412.svs
1022635.svs
1022740.svs
1022994.svs
1023024.svs
1027464.svs
1027479.svs
1027509.svs
1027511.svs
1027528.svs
132495-a PAS - 2021-02-17 14.18.56.ndpi
136023-a PAS - 2021-02-18 10.13.03.ndpi
136023-a PAS - 2021-02-18 10.15.26.ndpi
137052-a PAS - 2021-02-18 10.18.49.ndpi
137052-a PAS - 2021-02-18 10.20.10.ndpi
137388-a PAS - 2021-02-18 10.42.12.ndpi
137588 PAS - 2021-02-18 10.34.10.ndpi
137588 PAS - 2021-02-18 10.40.58.ndpi
Bela AR 001-01 PAS ADD.svs
Bela AR 001-01 PAS.svs
Bela AR 004-01 PAS ADD.svs
Bela AR 004-01 PAS.svs
Bela AR 004-02 PAS ADD.svs
Bela AR 004-02 PAS.svs
Bela AR 005-01 PAS ADD.svs
Bela AR 005-01 PAS.svs
Bela AR 006-01 PAS ADD.svs
Bela AR 006-01 PAS.svs
Bela AR 007-01 PAS ADD.svs
Bela AR 007-01 PAS.svs
Bela AR 008-01 PAS ADD.svs
Bela AR 008-01 PAS.svs
Bela AR 009-01 PAS ADD.svs
Bela AR 009-01 PAS.svs
Bela AR 010-01 PAS ADD.svs
Bela AR 010-01 PAS.svs
Bela AR 015-01 PAS ADD.svs
Bela AR 015-01 

In [5]:
belaFolderId = '5cf821711ee779b5180284fd'




In [87]:
len(allTxSlidesRaw)

6434

In [25]:
addlPasSlides = []

for s in allTxSlidesRaw:
    if 'pas' in s['name'].lower():
        if (s['name'] not in leePAS_set):
            addlPasSlides.append(s)
    # elif 'stain' in s['meta'].keys():
    #     print(s['meta'])
    elif 'PAS' in s['meta'].values():
        if (s['name'] not in leePAS_set):
            # print(s['meta'])
            addlPasSlides.append(s)


print(len(addlPasSlides))

433


In [5]:
import json
with open("fileDataForLee.json","w") as fp:
    json.dump(addlPASslides, fp)